In [ ]:
# storing data for mean, permutation entropy, and ordinal distribution of error function
from matplotlib import pyplot as plt
import numpy as np
import math
from scipy.integrate import solve_ivp
from scipy import integrate
from matplotlib.pyplot import figure
from itertools import repeat
import seaborn as sn
import ordpy
import pandas as pd
import array

alpha = 0
alpha_array_inner_layer = []
alpha_array_outer_layer = []
beta_array = []
probability_difference_list = []
PE_list = []
avg_error_func_array = []
beta = 0
alpha_max = 0.04
beta_max = 0.04
beta_repeat_times = int((beta_max-beta)/0.0005)

while beta <= beta_max:
    while alpha <= alpha_max:
        alpha_array_inner_layer.append(alpha)
        alpha += 0.0005
    beta_array.extend(repeat(beta,beta_repeat_times))
    alpha_array_outer_layer.extend(alpha_array_inner_layer)
    beta += 0.0005

# print(beta_array)
# print(alpha_array_outer_layer)
    
xi_0 = 0.175
T_1_ = 5
xi_P = 0.175
A_P = 0.01
omega_P = 0.7
A_0 = 0.5
omega_T_ = 0.7
c = 0.1
b_0 = 0.6
a = 0.7
T_2_ = 5
g = 0.1

def JJ (t, y, xi_0, T_1_, xi_P, A_P, omega_P, A_0, omega_T_, alpha, beta, c, b_0, a, T_2_, g):
    x1, y1, x2, y2, z = y
    dx1 = x1*(1-xi_0*math.e**(-1/T_1_)-xi_P)-y1-1/3*x1**3+A_P*math.cos(omega_P*t)+A_0*math.e**(-1/T_1_)*math.cos(omega_T_*t)-alpha*math.sin(z)-beta*(x1-x2)
    dy1 = c*(-b_0*math.e**(1/T_1_)*y1+a+x1)
    dx2 = x2*(1-xi_0*math.e**(-1/T_2_)-xi_P)-y2-1/3*x2**3+A_P*math.cos(omega_P*t)+A_0*math.e**(-1/T_2_)*math.cos(omega_T_*t)-alpha*math.sin(z)-beta*(x1-x2)
    dy2 = c*(-b_0*math.e**(1/T_2_)*y2+a+x2)
    dz = g*(x1-x2)
    dydt = [dx1, dy1, dx2, dy2, dz]
    return dydt

tStart = 0
tEnd = 8000

X0 = [0.2, 0.1, 0.02, 0.01, 0.001]

k = 0
while k < len(beta_array) and k < len(alpha_array_outer_layer):
    alpha = alpha_array_outer_layer[k]
    beta = beta_array[k]
    
    solution = integrate.solve_ivp(JJ, [tStart, tEnd], X0, method='RK45', t_eval=np.linspace(tStart,tEnd,100000), args=(xi_0, T_1_, xi_P, A_P, omega_P, A_0, omega_T_, alpha, beta, c, b_0, a, T_2_, g))
    
    t_sol = solution.t
    x1_sol = solution.y[0]
    y1_sol = solution.y[1]
    x2_sol = solution.y[2]
    y2_sol = solution.y[3]
    z_sol = solution.y[4]

    # error function
    error_func = []
    i = 0
    while i<len(t_sol):
        error_func.append(math.sqrt((x1_sol[i]-x2_sol[i])**2+(y1_sol[i]-y2_sol[i])**2))
        i += 1
    
    # P(pattern0123) - P(pattern3210)
    ordinal_distribution = ordpy.ordinal_distribution(error_func, dx=4, dy=1, taux=1, tauy=1, return_missing=False, tie_precision=None)
    probability_difference=ordinal_distribution[1][0]-ordinal_distribution[1][len(ordinal_distribution[1])-1]
    probability_difference_list.append(probability_difference)
    
    # permutation entropy
    PE = ordpy.permutation_entropy(error_func, dx=4, taux=1, base=2, normalized=True)
    PE_list.append(PE)
    
    # average of error function
    sum_error_func = 0
    i = 0
    while i<len(t_sol):
        sum_error_func += math.sqrt((x1_sol[i]-x2_sol[i])**2+(y1_sol[i]-y2_sol[i])**2)
        i += 1
    avg_error_func = sum_error_func/len(t_sol)
    avg_error_func_array.append(avg_error_func)
    
    k += 1

probability_difference_matrix = np.array_split(probability_difference_list, beta_repeat_times)
PE_matrix = np.array_split(PE_list, beta_repeat_times)
avg_error_func_array_for_matrix = np.array_split(avg_error_func_array, beta_repeat_times)


from numpy import savetxt
savetxt('probability difference matrix for 2D heatmap (0-0.04).csv', probability_difference_matrix_for_2D_heatmap_opened_2, delimiter=',')
savetxt('permutation entropy matrix for 2D heatmap (0-0.04).csv', permutation_entropy_matrix_for_2D_heatmap_opened_2, delimiter=',')
savetxt('mean error function matrix for 2D heatmap (0-0.04).csv', mean_error_function_matrix_for_2D_heatmap_opened_2, delimiter=',')

In [ ]:
# retrieving data
from numpy import loadtxt
probability_difference_matrix_for_2D_heatmap_opened_2 = loadtxt('probability difference matrix for 2D heatmap (0-0.04).csv', delimiter=',')
permutation_entropy_matrix_for_2D_heatmap_opened_2 = loadtxt('permutation entropy matrix for 2D heatmap (0-0.04).csv', delimiter=',')
mean_error_function_matrix_for_2D_heatmap_opened_2 = loadtxt('mean error function matrix for 2D heatmap (0-0.04).csv', delimiter=',')

In [ ]:
# plotting heatmap for mean of error function
import numpy as np
import seaborn as sn
import matplotlib.pyplot as plt

con_prob_dif = np.vstack(mean_error_function_matrix_for_2D_heatmap_opened_2)
sn.set(rc = {'figure.figsize':(150, 150)})
heatmap_prob_dif = sn.heatmap(data=con_prob_dif, annot=True, annot_kws={"size":20})
# tick labels
n_ticklabels = 0
list_ticklabels = []
while n_ticklabels < 0.04:
    list_ticklabels.append('{:,}'.format(round(n_ticklabels, 5)))
    n_ticklabels += 0.0005
heatmap_prob_dif.set_xticklabels(list_ticklabels, fontsize=25)
heatmap_prob_dif.set_yticklabels(list_ticklabels, fontsize=25)
heatmap_prob_dif.invert_yaxis()
heatmap_prob_dif.set_title('average of error function', fontsize=60)
heatmap_prob_dif.set_xlabel('alpha', fontsize=60)
heatmap_prob_dif.set_ylabel('beta', fontsize=60)
cbar = heatmap_prob_dif.collections[0].colorbar
cbar.ax.tick_params(labelsize=60)
plt.show()
print(heatmap_prob_dif)

In [ ]:
# plotting heatmap for permutation entropy of error function
import numpy as np
import seaborn as sn
import matplotlib.pyplot as plt

con_prob_dif = np.vstack(permutation_entropy_matrix_for_2D_heatmap_opened_2)
sn.set(rc = {'figure.figsize':(150, 150)})
heatmap_prob_dif = sn.heatmap(data=con_prob_dif, annot=True, annot_kws={"size":20})
# tick labels
n_ticklabels = 0
list_ticklabels = []
while n_ticklabels < 0.04:
    list_ticklabels.append('{:,}'.format(round(n_ticklabels, 5)))
    n_ticklabels += 0.0005
heatmap_prob_dif.set_xticklabels(list_ticklabels, fontsize=25)
heatmap_prob_dif.set_yticklabels(list_ticklabels, fontsize=25)
heatmap_prob_dif.invert_yaxis()
heatmap_prob_dif.set_title('permutation entropy', fontsize=60)
heatmap_prob_dif.set_xlabel('alpha', fontsize=60)
heatmap_prob_dif.set_ylabel('beta', fontsize=60)
cbar = heatmap_prob_dif.collections[0].colorbar
cbar.ax.tick_params(labelsize=60)
plt.show()
print(heatmap_prob_dif)

In [ ]:
# plotting heatmap for probability difference in ordinal distribution of error function
import numpy as np
import seaborn as sn
import matplotlib.pyplot as plt

con_prob_dif = np.vstack(probability_difference_matrix_for_2D_heatmap_opened_2)
sn.set(rc = {'figure.figsize':(150, 150)})
heatmap_prob_dif = sn.heatmap(data=con_prob_dif, annot=True, annot_kws={"size":20})
# tick labels
n_ticklabels = 0
list_ticklabels = []
while n_ticklabels < 0.04:
    list_ticklabels.append('{:,}'.format(round(n_ticklabels, 5)))
    n_ticklabels += 0.0005
heatmap_prob_dif.set_xticklabels(list_ticklabels, fontsize=25)
heatmap_prob_dif.set_yticklabels(list_ticklabels, fontsize=25)
heatmap_prob_dif.invert_yaxis()
heatmap_prob_dif.set_title('difference between probability of pattern0123 and pattern3210', fontsize=60)
heatmap_prob_dif.set_xlabel('alpha', fontsize=60)
heatmap_prob_dif.set_ylabel('beta', fontsize=60)
cbar = heatmap_prob_dif.collections[0].colorbar
cbar.ax.tick_params(labelsize=60)
plt.show()
print(heatmap_prob_dif)